Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-136443
aml-quickstarts-136443
southcentralus
61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-bike-experiment-1'
project_folder = './pipeline-bike-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-bike-experiment-1,quick-starts-ws-136443,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Creating
Succeeded................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Bikesharing Dataset"
description_text = "Bike Sharing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Azure/MachineLearningNotebooks/master/how-to-use-azureml/automated-machine-learning/forecasting-bike-share/bike-no.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,instant,season,yr,mnth,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500684,6.519836,2.997264,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500342,3.451913,2.004787,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,1.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,1.000000,7.000000,3.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,10.000000,5.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,6.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [6]:
dataset.take(5).to_pandas_dataframe()

,instant,date,season,yr,mnth,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,6,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,2,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,3,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'normalized_root_mean_squared_error',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "forecasting",
                             training_data=dataset,
                             time_column_name="date", 
                             label_column_name="cnt",  
                             path = project_folder,
                             enable_early_stopping= True,
                             #featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [8]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [9]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [10]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [11]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [9e625a34][dd66bc2a-a357-4481-a6b8-9d86d23acd78], (This step will run and generate new outputs)
Submitted PipelineRun e0c52c4c-c268-4d7b-b94b-f2b5c9d75b45
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-bike-experiment-1/runs/e0c52c4c-c268-4d7b-b94b-f2b5c9d75b45?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-136443/workspaces/quick-starts-ws-136443


In [12]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [13]:
pipeline_run.wait_for_completion()

PipelineRunId: e0c52c4c-c268-4d7b-b94b-f2b5c9d75b45
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-bike-experiment-1/runs/e0c52c4c-c268-4d7b-b94b-f2b5c9d75b45?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-136443/workspaces/quick-starts-ws-136443
PipelineRun Status: Running


StepRunId: 043c1450-93d8-4d2e-b2e4-fd113bcd2769
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-bike-experiment-1/runs/043c1450-93d8-4d2e-b2e4-fd113bcd2769?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-136443/workspaces/quick-starts-ws-136443
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '043c1450-93d8-4d2e-b2e4-fd113bcd2769', 'target': 'auto-ml', 'status': 'Completed', 'startTimeUtc': '2021-01-29T00:19:51.909504Z', 'endTimeUtc': '2021-01-29T00:59:14.053731Z', 'properties': {'ContentSnapshot

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [14]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/043c1450-93d8-4d2e-b2e4-fd113bcd2769/metrics_data, 1 files out of an estimated total of 1


In [15]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,043c1450-93d8-4d2e-b2e4-fd113bcd2769_2,043c1450-93d8-4d2e-b2e4-fd113bcd2769_8,043c1450-93d8-4d2e-b2e4-fd113bcd2769_12,043c1450-93d8-4d2e-b2e4-fd113bcd2769_29,043c1450-93d8-4d2e-b2e4-fd113bcd2769_35,043c1450-93d8-4d2e-b2e4-fd113bcd2769_39,043c1450-93d8-4d2e-b2e4-fd113bcd2769_15,043c1450-93d8-4d2e-b2e4-fd113bcd2769_40,043c1450-93d8-4d2e-b2e4-fd113bcd2769_42,043c1450-93d8-4d2e-b2e4-fd113bcd2769_31,...,043c1450-93d8-4d2e-b2e4-fd113bcd2769_3,043c1450-93d8-4d2e-b2e4-fd113bcd2769_11,043c1450-93d8-4d2e-b2e4-fd113bcd2769_22,043c1450-93d8-4d2e-b2e4-fd113bcd2769_28,043c1450-93d8-4d2e-b2e4-fd113bcd2769_27,043c1450-93d8-4d2e-b2e4-fd113bcd2769_34,043c1450-93d8-4d2e-b2e4-fd113bcd2769_0,043c1450-93d8-4d2e-b2e4-fd113bcd2769_19,043c1450-93d8-4d2e-b2e4-fd113bcd2769_23,043c1450-93d8-4d2e-b2e4-fd113bcd2769_37
normalized_mean_absolute_error,[0.13336401288541186],[4.2470706345303317e-07],[0.007148386182616437],[0.002179663425762306],[0.00012979110982878092],[0.04407040765646379],[0.02233435541281518],[0.00014740399589849615],[1.5409804953820608e-07],[0.00026260011750575565],...,[0.1097100782328578],[0.02072845342518605],[0.024188663249259876],[0.01685004466581847],[0.02127882573288761],[4.2470706340594703e-07],[0.07953388438319364],[0.024188663249259876],[0.015326225903768465],[0.00010647384107460038]
root_mean_squared_log_error,[0.6985293680914578],[1.9691822686240814e-06],[0.03452643582075563],[0.011105248428247805],[0.0006019018422882994],[0.1458221554733587],[0.08113129445075362],[0.0006827411746728629],[7.149066021838734e-07],[0.0014572031784441108],...,[0.43670407984373905],[0.09391663626613908],[0.10317703711885215],[0.06584018763136737],[0.08791468090818544],[1.96918226826881e-06],[0.31186772705979476],[0.10317703711885215],[0.06653708755868148],[0.00038413995973787254]
spearman_correlation,[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],...,[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0],[-1.0]
normalized_root_mean_squared_log_error,[0.11765088085262347],[3.316625456936514e-07],[0.0058151679407724335],[0.0018704185097332972],[0.00010137624152512885],[0.024560320328611617],[0.01366466277992622],[0.00011499173014598333],[1.2040924163875155e-07],[0.00024543168169666077],...,[0.07355255485668191],[0.01581805359682461],[0.01737775082236407],[0.011089234646635755],[0.0148071650544707],[3.316625456338143e-07],[0.052526800552910216],[0.01737775082236407],[0.011206611086424755],[6.469936225778665e-05]
mean_absolute_error,[1159.2],[0.003691553795533764],[62.133772699302064],[18.945634496725962],[1.1281443266317637],[383.05998334998327],[194.13021724818958],[1.2812355323497286],[0.001339420246586087],[2.282520221360028],...,[953.6],[180.17171717171715],[210.24786096256685],[146.46058823529415],[184.95555327025914],[0.0036915537951244916],[691.3085230587192],[210.24786096256685],[133.2155555555555],[0.9254706266204267]
median_absolute_error,[1159.2],[0.003691553795533764],[62.133772699302064],[18.945634496725962],[1.1281443266317637],[383.05998334998327],[194.13021724818958],[1.2812355323497286],[0.001339420246586087],[2.282520221360028],...,[953.6],[180.17171717171715],[210.24786096256685],[146.46058823529415],[184.95555327025914],[0.0036915537951244916],[691.3085230587192],[210.24786096256685],[133.2155555555555],[0.9254706266204267]
r2_score,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
normalized_root_mean_squared_error,[0.13336401288541186],[4.2470706345303317e-07],[0.007148386182616437],[0.002179663425762306],[0.00012979110982878092],[0.04407040765646379],[0.02233435541281518],[0.00014740399589849615],[1.5409804953820608e-07],[0.00026260011750575565],...,[0.1097100782328578],[0.02072845342518605],[0.024188663249259876],[0.01685004466581847],[0.02127882573288761],[4.2470706340594703e-07],[0.07953388438319364],[0.024188663249259876],[0.015326225903768465],[0.00010647384107460038]
root_mean_squared_error,[1159.2],[0.003691553795533764]

### Retrieve the Best Model

In [16]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/043c1450-93d8-4d2e-b2e4-fd113bcd2769/model_data, 1 files out of an estimated total of 1


In [17]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

ForecastingPipelineWrapper(pipeline=Pipeline(memory=None,
                                             steps=[('timeseriestransformer',
                                                     TimeSeriesTransformer(featurization_config=None,
                                                                           pipeline_type=<TimeSeriesPipelineType.FULL: 1>)),
                                                    ('prefittedsoftvotingregressor',
                                                     PreFittedSoftVotingRegressor(estimators=[('25',
                                                                                               Pipeline(memory=None,
                                                                                                        steps=[('standardscalerwrapper',
                                                                                                                <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7febb

In [18]:
best_model.steps

[('timeseriestransformer',
  TimeSeriesTransformer(featurization_config=None,
                        pipeline_type=<TimeSeriesPipelineType.FULL: 1>)),
 ('prefittedsoftvotingregressor',
  PreFittedSoftVotingRegressor(estimators=[('25',
                                            Pipeline(memory=None,
                                                     steps=[('standardscalerwrapper',
                                                             <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7febb7f23470>),
                                                            ('lassolars',
                                                             LassoLars(alpha=0.001,
                                                                       copy_X=True,
                                                                       eps=2.220446049250313e-16,
                                                                       fit_intercept=True,
                        

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [19]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bikesharing Train", description="Training bikesharing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bikesharing Train,80c32f23-9bfa-4c9c-93bc-76860de4a089,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [20]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [21]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-bike-rest-endpoint"}
                        )

In [22]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  9035afe0-4527-4e1c-ab4b-423ac90c83d7


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [23]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-bike-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …